# Objectives
## 1. Determine the industry factors that are most important in predicting the salary amounts for these data.
## 2. Determine the factors that distinguish job categories and titles from each other. For example, can required skills accurately predict job title?


# Scope
## I am limiting my search to fulltime data-related job postings within Sydney

### Borrowing the scraping techniques from https://jessesw.com/Data-Science-Skills/:

In [2]:

from bs4 import BeautifulSoup # For HTML parsing
from urllib.request import urlopen # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
%matplotlib inline


#### From inspection,
#### class="hOyLUJnCRhx YDKfqI6z" denotes the 'sponsored' job postings
#### and class = " row result" gives normal job postings

In [2]:
# use requests to copy html and see if the URL works
import requests
test = requests.get("https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time")
test
print(test.content[:200])

b'<!doctype html>\n<html lang="en">\n  <head>\n    <script>\n      (function(l,d,j,t){\n        d.test(l.hash) && j.test(l.pathname) && (l.href = l.pathname + (l.search ? l.search + \'&\' : \'?\') + l.hash.slice'


In [3]:
soup = BeautifulSoup(test.content, 'html.parser')
BASE_URL = "https://www.seek.com.au"

#body = soup.find_all("a","_1OFaluu")

#for row in soup.find_all("a","_1OFaluu"):
#    print(row.get('href'))
#    print(BASE_URL + row.get('href'))
    
#### OKAY. FINALLY GOT THE LINKS FOR EACH JOB ENTRY ON THE PAGE


In [4]:
# getting job links, function adapted from http://www.gregreda.com/2013/03/03/web-scraping-101-with-python/:

#BASE_URL = "http://www.chicagoreader.com"

#def get_category_links(section_url):
#    html = urlopen(section_url).read()
#    soup = BeautifulSoup(html, "lxml")
#    boccat = soup.find("dl", "boccat")
#    category_links = [BASE_URL + dd.a["href"] for dd in boccat.findAll("dd")]
#    return category_links

BASE_URL = "https://www.seek.com.au"
test_links=[]
def get_job_links(section_url):
    html = requests.get(section_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    for row in soup.find_all("a","_1OFaluu"):
        test_links.append(BASE_URL+row.get('href'))
    return test_links
    

In [5]:
sample = get_job_links('https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time')
print(test_links)

['https://www.seek.com.au/job/33426869?type=standout&tier=no_tier&pos=1&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33427105?type=standard&tier=no_tier&pos=2&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33427108?type=standard&tier=no_tier&pos=3&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33427070?type=standard&tier=no_tier&pos=4&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33427068?type=standard&tier=no_tier&pos=5&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33423293?type=standard&tier=no_tier&pos=6&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0430460&ref=beta', 'https://www.seek.com.au/job/33423239?type=standard&tier=no_tier&pos=7&whereid=1000&userqueryid=81e13a753f7e2cf

#### Okay, now I've gotten the job urls from one page. now to figure out how to get them from multiple pages

In [6]:
## Url for page 1:
#https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time
## Url for page 2:
#https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time?page=2
## Url for page 3:
#https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time?page=3

#### Sweet, looks like I just have to add "?page=x" to the url, where x is the number of pages I want. Can use range function to iterate.
#### But how many pages to get 1000 job postings?

In [7]:
print(len(test_links))

20


#### 20 links from one page, that means 50 pages. Let's make it 55 to have a 10% margin.

In [12]:
# Initialise job_links list
job_links = []

## function to collect links from one page.
def get_job_links(section_url):
    html = requests.get(section_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    for row in soup.find_all("a","_1OFaluu"):
        job_links.append(BASE_URL+row.get('href'))
    return job_links

## page-turner
for page in range(2,56):
    ## run function for that page
    get_job_links('https://www.seek.com.au/data-jobs/in-All-Sydney-NSW/full-time?page=%s' %page)


print(job_links)

['https://www.seek.com.au/job/33418644?type=standard&tier=no_tier&pos=21&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33412792?type=standard&tier=no_tier&pos=22&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33412489?type=standard&tier=no_tier&pos=23&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33410850?type=standard&tier=no_tier&pos=24&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33409374?type=standard&tier=no_tier&pos=25&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33406812?type=standard&tier=no_tier&pos=26&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-0552252&ref=beta', 'https://www.seek.com.au/job/33406422?type=standout&tier=no_tier&pos=27&whereid=1000&userqueryid=81e13a75

In [13]:
print(len(job_links))

1080


In [14]:
# test parts of my get_job_info function here
html = requests.get('https://www.seek.com.au/job/33412792?type=standard&tier=no_tier&pos=1&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-1962949&ref=beta')
soup = BeautifulSoup(html.content, 'html.parser')
title = soup.find("h1","jobtitle").string
salary = soup.find("div", {'itemprop':"baseSalary"}).renderContents()
location = soup.find("span", {'data-automation':'apply_text_location_area'}).renderContents()
industry = soup.find("span", {'itemprop':'industry'}).renderContents()
description = soup.find("div", {'class':'templatetext'}).renderContents()
print(title)
print(salary)
print(location)
print(industry)
print(description)

Data Engineer / Data Scientist
    
b'Start up culture'
b'Sydney'
b'Information &amp; Communication Technology'
b"\n<p>Australian start up success story requires a data specialist to join their Sydney Team. You will be working within a team of Data expects providing the next generation of algorithms to address today's and tomorrow's challenges.</p>\n<p><strong>Responsibilities</strong></p>\n<ul>\n<li>Apply your expertise in quantitative analysis, data mining, and the presentation of data to see beyond the numbers and understand how our users interact with our core products\r\n</li><li>Partner with Product and Engineering teams to solve problems and identify trends and opportunities\r\n</li><li>Inform, influence, support, and execute our product decisions </li><li>Build/maintain reports, dashboards, and metrics to monitor performance\xc2\xa0 </li><li>Mine massive amounts of data and extract useful product insights</li></ul>\n<p><strong>Experience</strong></p>\n<p>\xc2\xa0</p>\n<ul>\n<li

In [26]:
# Okay, now to test the above code on another job link and see if it works
html = requests.get('https://www.seek.com.au/job/33406422?type=standout&tier=no_tier&pos=8&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-1962949&ref=beta')
soup = BeautifulSoup(html.content, 'html.parser')
title = soup.find("h1","jobtitle").string
# to take into account cases where salary is not listed, I want to write
#salary = NaN if soup.find("div", {'itemprop':"baseSalary"}).renderContents() not valid
try:
    salary = soup.find("div", {'itemprop':"baseSalary"}).renderContents()
except Exception:
    salary = 'NaN'
    
location = soup.find("span", {'data-automation':'apply_text_location_area'}).renderContents()
industry = soup.find("span", {'itemprop':'industry'}).renderContents()
description = soup.find("div", {'class':'templatetext'}).renderContents()
print(title)
print(salary)
print(location)
print(industry)
print(description)

Data Steward - SAP / MDM
      
b'$70,000 to $80,000 + super'
b'Sydney'
b'Information &amp; Communication Technology'
b"Data Steward - SAP / MDM<br/>\n<br/>\r\nWe are looking for Data Steward with good exposure using MDM to join a well known technical company based in Ryde. This role will give you outstanding commercial and internal growth for your career.<br/>\n<br/>\n<strong>Role:</strong><br/>\n<br/>\n<ul>\n<li>Building positive working relationships across the business, in particular with data stewards and owners\r\n</li><li>Ensure established data management processes and procedures are being followed accurately and efficiently\r\n</li><li>Prepare data monitoring reports to identify data quality and completeness issues across the enterprise system landscape\r\n</li><li>Take ownership of data quality exceptions and ensure that these are resolved in line with cross functional and entity data requirements\r\n</li><li>Continuous improvement of processes and procedures, highlighting in

In [86]:
# Test 2
html = requests.get('https://www.seek.com.au/job/33411398?type=standout&tier=no_tier&pos=5&whereid=1000&userqueryid=81e13a753f7e2cfe6febebc0650ea597-1962949&ref=beta')
soup = BeautifulSoup(html.content, 'html.parser')
title = soup.find("h1","jobtitle").string
# to take into account cases where salary is not listed, I want to write
#salary = NaN if soup.find("div", {'itemprop':"baseSalary"}).renderContents() not valid
try:
    salary = soup.find("div", {'itemprop':"baseSalary"}).renderContents()
except Exception:
    salary = 'NaN'
    
location = soup.find("span", {'data-automation':'apply_text_location_area'}).renderContents()
advertiser = soup.find("span", {'class':'state-message'}).renderContents()
industry = soup.find("span", {'itemprop':'industry'}).renderContents()
description = soup.find("div", {'class':'templatetext'}).renderContents()
print(title)
print(salary)
print(location)
print(advertiser)
print(industry)
print(description)

Data Administrator, product comparison
      
NaN
b'Sydney'
b'Mozo Pty Ltd - '
b'Administration &amp; Office Support'
b"\n<p>Is <em><strong>this</strong></em> you:</p>\n<ul>\n<li>You genuinely enjoy spending your days on regular, familiar tasks and working with details.\r\n</li><li>You take great pride in accuracy and efficiency. </li><li>You can calmly deal with large volumes of numbers and facts. </li><li>You can read and comprehend detailed documents and extract the key information.\r\n</li><li>You are comfortable with software including spreadsheets. </li><li>You like to work independently, can organise yourself and prioritise tasks. </li><li>You will commit to a job for at least a couple of years. </li><li>You are a good communicator. </li><li>You are a morning person.</li></ul>\n<p>If all of those statements describe you, we'd like to hear from you.\xc2\xa0</p>\n<p><strong>NOTE: To be considered for this role, you must send us your resum\xc3\xa9 AND a separate cover letter of at 

In [45]:
## test adding stuff to dataframe
#testdf = pd.DataFrame(columns=['unos','dos','tres'])
#print(testdf.head())

Empty DataFrame
Columns: [unos, dos, tres]
Index: []


In [50]:
#testdf.loc[0] = [1,2,3]
#one = 8
#two = 9
#three = 10
#testdf.loc[len(testdf.unos)]=[one,two,three]

#print(testdf.head())

   unos  dos  tres
0   1.0  2.0   3.0
1   4.0  5.0   6.0
2   4.0  5.0   6.0
3   8.0  9.0  10.0


In [117]:

# now i'm going to write a function to get job info. I want: Title, Salary (where available), Location, Industry,
# Description
# 
# using the below function from http://www.gregreda.com/2013/03/03/web-scraping-101-with-python/
#def get_category_winner(category_url):
#    html = urlopen(category_url).read()
#    soup = BeautifulSoup(html, "lxml")
#    category = soup.find("h1", "headline").string
#    winner = [h2.string for h2 in soup.findAll("h2", "boc1")]
#    runners_up = [h2.string for h2 in soup.findAll("h2", "boc2")]
#    return {"category": category,
#            "category_url": category_url,
#            "winner": winner,
#            "runners_up": runners_up}

## initialize DataFrame
data = pd.DataFrame(columns=['title','salary','location','advertiser', 'industry','description'])

def get_job_info(job_url):
    html = requests.get(job_url)
    soup = BeautifulSoup(html.content, 'html.parser')
    # Get the row
    try:
        title = soup.find("h1","jobtitle").string
    except Exception:
        title = 'NaN'
    # to take into account cases where salary is not listed, I want to write
    #salary = NaN if soup.find("div", {'itemprop':"baseSalary"}).renderContents() not valid
    try:
        salary = soup.find("div", {'itemprop':"baseSalary"}).renderContents()
    except Exception:
        salary = 'NaN'
    
    try:
        location = soup.find("span", {'data-automation':'apply_text_location_area'}).renderContents()
    except Exception:
        location = 'NaN'
    try:
        advertiser = soup.find("span", {'class':'state-message'}).renderContents()
    except Exception:
        advertiser = 'NaN'
    try:
        industry = soup.find("span", {'itemprop':'industry'}).renderContents()
    except Exception:
        industry = 'NaN'
    try:
        description = soup.find("div", {'class':'templatetext'}).renderContents()
    except Exception:
        description = 'NaN'
    # add to data dataframe
    data.loc[len(data.title)]=[title,salary,location,advertiser,industry,description]
    # add a counter so I can see what's happenning
    if len(data.title) >0:
        print(len(data.title))

In [118]:

## iterate through each url in job_links
for job in job_links:
    ## run function for that job
    get_job_info(job)
    ## pray.

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [120]:
## write my dataframe to a csv!!!
data.to_csv('/Users/lohyenwei/Desktop/data_science/Projects/Project4/Project4_uploads/yen/dataP4.csv')

In [48]:
data = pd.read_csv('/Users/lohyenwei/Desktop/data_science/Projects/Project4/Project4_uploads/yen/dataP4.csv')
print(data.head())
print(data.columns)

   Unnamed: 0                                       title  \
0           0  Project Coordinator - Data entry\r\n         
1           1      Data Engineer / Data Scientist\r\n       
2           2                      Data Analyst\r\n         
3           3                Media Data Analyst\r\n         
4           4                                         NaN   

                       salary   location                 advertiser  \
0   b'Up to $42ph plus Super'  b'Sydney'  b'Preacta Recruitment - '   
1         b'Start up culture'  b'Sydney'   b'Motion Recruitment - '   
2                         NaN  b'Sydney'        b'Westpac Group - '   
3                         NaN  b'Sydney'     b'Big Wave Digital - '   
4  b'Attractive remuneration'  b'Sydney'                  b'IPA - '   

                                        industry  \
0         b'Administration &amp; Office Support'   
1  b'Information &amp; Communication Technology'   
2            b'Banking &amp; Financial Services'  

In [49]:
# drop the Unnamed column.
data.drop('Unnamed: 0', axis=1, inplace=True)
print(data.isnull().sum())

title          147
salary         677
location         3
advertiser       3
industry         3
description     26
dtype: int64


In [50]:
# fill nulls
data.title.fillna(value="NA", inplace=True)
data.salary.fillna(value=0, inplace=True)
data.location.fillna(value="NA", inplace=True)
data.advertiser.fillna(value="NA", inplace=True)
data.industry.fillna(value="NA", inplace=True)
data.description.fillna(value="NA", inplace=True)
print(data.isnull().sum())

title          0
salary         0
location       0
advertiser     0
industry       0
description    0
dtype: int64


In [51]:
print(data.dtypes)
print('Shape', data.shape)

title          object
salary         object
location       object
advertiser     object
industry       object
description    object
dtype: object
Shape (1080, 6)


In [52]:
print(data.title.value_counts())

NA                                                                                           147
Data Scientist\r\n                                                                             8
Customer Support Officer - Multiple Positions - State Government Role\r\n                      6
Business Support Manager\r\n                                                                   6
Administration Supervisor (Operational Support role)\r\n                                       5
Data Scientist\r\n                                                                             5
Business Development Manager\r\n                                                               5
Data Analyst\r\n                                                                               4
Modeller - 3D CAD \r\n                                                                         4
Senior Analyst - Big Data\r\n                                                                  3
Data Analyst\r\n              

In [54]:
# clean up titles
# opioids.Generic_Name = opioids.Generic_Name.map(lambda x: x.replace('-', '_'))
data.title = data.title.map(lambda x: x.replace('\r\n', ''))
data.title.value_counts()

NA                                                                                  147
Data Scientist                                                                        8
Customer Support Officer - Multiple Positions - State Government Role                 6
Business Support Manager                                                              6
Data Scientist                                                                        5
Administration Supervisor (Operational Support role)                                  5
Business Development Manager                                                          5
Modeller - 3D CAD                                                                     4
Data Analyst                                                                          4
Data Analyst                                                                          3
Accountant                                                                            3
Payroll Officer                 

In [55]:
data.salary.value_counts()

0                                                        677
b'$60,000 - $79,999'                                       8
b'$45,000 - $49,999'                                       4
b'$40 - $50 p.h. + Super '                                 4
b'End User Company - Great Package, Bonuses + Agile'       4
b'$75,000 + super + bonus'                                 3
b"Approx $51,375 FTE incl super + salary pack'ing"         3
b'Attractive salary + 12% Super + Leave loading!'          3
b'$65,000 - $69,999'                                       3
b'$45000.00 - $55000.00 p.a. + Super'                      2
b'Strong daily rates upto $950!!'                          2
b'Enjoy working  in a flexible environment!'               2
b'$92,026'                                                 2
b'$60,000 - $74,999'                                       2
b'Highly competitive salary and benefits'                  2
b'Super'                                                   2
b'$85,000 - $99,999'    

In [56]:
# clean salary
# Keeping the numbers and 'K'
def clean_salary(salary):
    sal=''
    count=0
    prev_x=''
    for x in salary:
        if x=='.':
            count=3
        if count==0:
            if x.isdigit():
                sal+=x
            elif x.lower()=='k':
                if prev_x.isdigit():
                    sal+='K'
        else:
            count-=1
        prev_x=x
    return sal

In [57]:
data['cleaned_salary'] = [clean_salary(sal) if sal != 0 else 0 for sal in data['salary']]

In [59]:
data['cleaned_salary'].value_counts()

0                   677
                    109
6000079999            8
12                    6
140K                  4
4050                  4
6500070000            4
4500049999            4
51375                 3
110000130000          3
6500069999            3
90000120000           3
65000                 3
80K85K                3
6000064999            3
75000                 3
6000074999            3
110000120000          2
111677132809          2
6724873635            2
120000                2
130K150K              2
140K160K150K300K      2
750                   2
145000                2
80000119999           2
6070K                 2
950                   2
8500099999            2
9123116137889         2
                   ... 
660                   1
9001200               1
6075K                 1
110000                1
7000075000            1
120K140K              1
80K130K               1
80000100000           1
100K900K              1
150K180K10            1
4000054999      

In [60]:
data.cleaned_salary = data.cleaned_salary.map(lambda x: 0 if (x=='') or (x=='0K') or (x=='0') else x)

In [61]:
# Eliminating if more than one K is together and then replacing k with '000' 
def replace_K(cleaned_salary):
    sal=''
    prev_x=''
    for x in cleaned_salary:
        if x.upper()=='K':
            if prev_x.upper()=='K':
                pass
            else:
                sal+='000'
        else:
            sal+=x
        prev_x=x
    return sal

In [63]:
data['number_salary'] = [replace_K(sal) if sal != 0 else 0 for sal in data['cleaned_salary']]
data['number_salary'].value_counts()

0                  787
6000079999           8
12                   6
65000                5
90000120000          5
4050                 4
6500070000           4
8000085000           4
5000055000           4
140000               4
75000                4
120000               4
4500049999           4
6000064999           3
90000100000          3
6000074999           3
55000                3
110000130000         3
50000                3
6500069999           3
4500050000           3
51375                3
70000                3
130000150000         3
80000119999          2
950                  2
160000180000         2
4000044999           2
130000140000         2
85000100000          2
                  ... 
130000159999         1
2527                 1
150000180000         1
110000139999         1
110000150000         1
80000130000          1
130000189999         1
7075000              1
180000300000         1
100110000            1
14500016500030       1
75000100000          1
10000090000

In [66]:
# from inspecting the data, if there is a 2-digit number < 30, it is not a salary.
# 3-digit numbers are daily rates 
# 4-digit numbers refer to a range of hourly rates (e.g, 3040 = $30-$40 per hour)
# 6-digit numbers refer to a range of annual rates (e.g, 120140 = $120-$140 thousand per year)
# unless it's just straight annual salary (e.g 110000 = $110,000 per year)
# or also a possible range of daily rates
# then you have the 11-digit numbers, which are a range of annual rates (e.g 70000100000 = $70 - $100,000 per year)
# ...and I have what looks like a phone number. 0412050866. Gonna drop that.
data = data[data.number_salary != '0412050866']
data['number_salary'].value_counts()


0                  787
6000079999           8
12                   6
65000                5
90000120000          5
120000               4
4050                 4
75000                4
5000055000           4
8000085000           4
6500070000           4
4500049999           4
140000               4
50000                3
6500069999           3
110000130000         3
51375                3
4500050000           3
90000100000          3
6000074999           3
70000                3
130000150000         3
55000                3
6000064999           3
130000140000         2
110000120000         2
101827121332         2
5500060000           2
80000119999          2
5055000              2
                  ... 
8500014000           1
120000140000         1
100000149999         1
100000140000         1
710009500017         1
70000120000          1
80000100000          1
7000085000           1
150000170000         1
42                   1
130000               1
14500016500030       1
100110000  

In [121]:
## write a function to turn all salary numbers to annual salary.

def final_salary_clean(salary):
    salary = str(salary)
    if len(salary)<2:
        salary=0
    elif len(salary)<3:
        if float(salary)<30:
            salary=0
        else:
            salary= float(salary)*38*52 #38 hours a week, 52 weeks a year
    elif len(salary)<5:
        if float(salary)<200:
            pass
        else:
            hourly_rate=(float(salary[0:2])+float(salary[2:4]))/2
            salary=hourly_rate*38*52
    elif len(salary)<7:
        if float(salary)>250000:
            daily_rate=(float(salary[0:3])+float(salary[3:6]))/2
            salary=daily_rate*260 #260 working days a year, assuming 5 days a week for 52 weeks
    elif len(salary)<8:
        daily_rate=(float(salary[0:3])+float(salary[3:]))/2
        salary=daily_rate*260
    elif len(salary)<13:
        salary=(float(salary[0:int(len(salary)/2)])+float(salary[int(len(salary)/2):]))/2
    # specific cases that don't fall under above cases
    elif salary=='601000004560000':
        salary=float(60000+100000+45000+60000)/4
    elif salary=='14500016500030':
        salary=float(145000+165000)/2
    elif salary=='15000018000010':
        salary=float(150000+180000)/2
    elif salary=='140000160000150000300000':
        salary = float(140000+160000+150000+300000)/4
    elif salary=='6000065000082709770': # the last 8 numbers are a phone number. ignore
        salary = float(60000+65000)/2
    elif salary=='750008500015000': # the last 5 numbers refer to a bonus. Ignore.
        salary = float(75000+85000)/2
    elif salary=='456000060100000':
        salary = float(45000+60000+100000)/3
    elif salary=='54750450005000':
        salary = float(45000) # in this example, 45k is base pay, 54.75k is salary + bonus + super, just take base pay
    elif salary=='9123116137889':
        salary = float(123116+137889)/2 # here, the leading 9 refers to Transport Service Grade 9
    elif salary=='1157012278017':
        salary = float(115700+122780)/2 # the last 17 refers to 17% super, and the 115700 is missing a 0 in the column
    else:
        pass
    return salary

In [122]:
data['final_salary'] = [final_salary_clean(sal) if sal != 0 else 0 for sal in data['number_salary']]

In [123]:
data.final_salary = data.final_salary.map(lambda x: int(x))

In [124]:
# checker to help me find salaries not covered in my function. any number with less than 13 digits 
# is covered in my function
data.final_salary.max()

741910

In [120]:
# checker to help me find salaries not covered in my function
data.loc[data['final_salary'] == 1157012278017]

,title,salary,location,advertiser,industry,description,cleaned_salary,number_salary,final_salary
1045,Engagement and Culture Lead,"b'Base: $115,70 - $122,780 + 17% super'",b'Sydney',b'University of New South Wales - ',b'Education &amp; Training',b'\n<ul>\n<li><strong>One of Australia\'s lead...,1157012278017,1157012278017,1157012278017


In [127]:
data.final_salary.value_counts()

0         793
120000      8
69999       8
140000      7
88920       7
105000      6
67499       6
75000       6
95000       6
47499       5
65000       5
100000      5
50000       5
82500       5
80000       5
55000       5
85000       5
160000      4
52500       4
62499       4
70000       4
67500       4
155000      3
52499       3
62500       3
60000       3
130000      3
47500       3
90000       3
92499       3
         ... 
107500      1
49999       1
74999       1
56560       1
82992       1
97500       1
54340       1
40013       1
210000      1
79040       1
146250      1
109999      1
175500      1
59999       1
73112       1
273000      1
185000      1
13000       1
66250       1
105170      1
139999      1
46436       1
695013      1
68333       1
54000       1
117500      1
94999       1
5500        1
154999      1
46000       1
Name: final_salary, dtype: int64

In [200]:
# consider binning salaries for easier processing. (e.g good<80,000 high>80,000)
def salary_categoriser(salary):
    salary = float(salary)
    if salary == 0:
        pass
    elif salary < 80000:
        salary = 'good'
    elif salary >= 80000:
        salary = 'high'
    return salary

In [201]:
data['sal_cat'] = [salary_categoriser(sal) for sal in data['final_salary']]

In [202]:
data.sal_cat.value_counts()

0.0     789
high    151
good    135
Name: sal_cat, dtype: int64

In [203]:
data.columns

Index(['title', 'salary', 'location', 'advertiser', 'industry', 'description',
       'cleaned_salary', 'number_salary', 'final_salary', 'sal_cat',
       'lead_count'],
      dtype='object')

In [204]:
data.location.value_counts()

Sydney    1075
Name: location, dtype: int64

In [205]:
# Brisbane? drop that, and the NA.
data = data[data.location != "b'Brisbane'"]
data = data[data.location != 'NA']

In [206]:
data.shape

(1075, 11)

In [207]:
data.location = data.location.map(lambda x: x.replace('b', ''))
data.location = data.location.map(lambda x: x.replace("'", ''))

In [208]:
data.location.value_counts()

Sydney    1075
Name: location, dtype: int64

In [209]:
data.advertiser.value_counts()

Private Advertiser                                      29
Perigon Group Pty Limited                               11
University of New South Wales                           11
HOBAN Recruitment                                       11
Correlate Resources                                     11
Bluefin Resources Pty Limited                           10
Finite IT Recruitment Solutions                          9
Preacta Recruitment                                      9
Data Talent Pty Ltd                                      9
Peopleank Australia NSW                                  8
Salient Group                                            8
Dimension Data                                           8
Six Degrees Executive Pty Ltd                            7
The University of Sydney                                 7
Mars Recruitment                                         7
Roert Walters                                            7
Infinity Pro                                            

In [210]:
data.advertiser = data.advertiser.map(lambda x: x.replace('b', ''))
data.advertiser = data.advertiser.map(lambda x: x.replace("'", ''))
data.advertiser = data.advertiser.map(lambda x: x.replace('-', ''))

In [211]:
data.advertiser.value_counts()

Private Advertiser                                      29
Perigon Group Pty Limited                               11
University of New South Wales                           11
HOBAN Recruitment                                       11
Correlate Resources                                     11
Bluefin Resources Pty Limited                           10
Finite IT Recruitment Solutions                          9
Preacta Recruitment                                      9
Data Talent Pty Ltd                                      9
Peopleank Australia NSW                                  8
Salient Group                                            8
Dimension Data                                           8
Six Degrees Executive Pty Ltd                            7
The University of Sydney                                 7
Mars Recruitment                                         7
Roert Walters                                            7
Infinity Pro                                            

In [212]:
data.industry.value_counts()

Information & Communication Technology    327
Administration & Office Support           126
Marketing & Communications                 82
Accounting                                 71
Sales                                      69
Banking & Financial Services               60
Science & Technology                       33
Manufacturing, Transport & Logistics       28
Trades & Services                          28
Call Centre & Customer Service             28
Government & Defence                       28
Advertising, Arts & Media                  24
Human Resources & Recruitment              23
Consulting & Strategy                      21
Education & Training                       20
Healthcare & Medical                       17
Engineering                                14
Retail & Consumer Products                 13
Real Estate & Property                     13
Insurance & Superannuation                 12
Design & Architecture                      11
Construction                      

In [213]:
data.industry = data.industry.map(lambda x: x.replace('b', ''))
data.industry = data.industry.map(lambda x: x.replace("'", ''))
data.industry = data.industry.map(lambda x: x.replace('amp;', ''))

In [214]:
data.industry.value_counts()

Information & Communication Technology    327
Administration & Office Support           126
Marketing & Communications                 82
Accounting                                 71
Sales                                      69
Banking & Financial Services               60
Science & Technology                       33
Manufacturing, Transport & Logistics       28
Trades & Services                          28
Call Centre & Customer Service             28
Government & Defence                       28
Advertising, Arts & Media                  24
Human Resources & Recruitment              23
Consulting & Strategy                      21
Education & Training                       20
Healthcare & Medical                       17
Engineering                                14
Retail & Consumer Products                 13
Real Estate & Property                     13
Insurance & Superannuation                 12
Design & Architecture                      11
Construction                      

In [215]:
data.description.value_counts()

NA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [216]:
# do I need to clean description? Will only be using it as a predictor based on occurences of words 
# like "lead", "leader", "Manager", "Director", "Senior", "executive", "director", "chief", "president", "Scientist"
# make a "lead" binary column and indicate 1 if "lead" or "leader" occurs
def lead_count(desc):
    count = data.description.str.contains(r'skills').sum()
#    for d in desc:
#        if d == 'exciting':
#            count += 1
#        elif d =='leader':
#            count +=1
#        else:
#            pass
    return count

In [217]:
data['lead_count'] = [lead_count(desc) for desc in data['description']]

In [218]:
data.lead_count.value_counts()

793    1075
Name: lead_count, dtype: int64

In [219]:
data

,title,salary,location,advertiser,industry,description,cleaned_salary,number_salary,final_salary,sal_cat,lead_count
0,Project Coordinator - Data entry,b'Up to $42ph plus Super',Sydney,Preacta Recruitment,Administration & Office Support,"b'<div class=""videoembed"" id=""FlashContent""></...",42,42,82992,high,793
1,Data Engineer / Data Scientist,b'Start up culture',Sydney,Motion Recruitment,Information & Communication Technology,"b""\n<p>Australian start up success story requi...",0,0,0,0,793
2,Data Analyst,0,Sydney,Westpac Group,Banking & Financial Services,b'<span>The Customer Data Analyst will support...,0,0,0,0,793
3,Media Data Analyst,0,Sydney,Big Wave Digital,Science & Technology,b'\n<div>\n<p>Are you looking to join massive ...,0,0,0,0,793
4,NA,b'Attractive remuneration',Sydney,IPA,Information & Communication Technology,b'<span><strong>Surveyor - Telecommunications<...,0,0,0,0,793
5,NA,"b'$90,000 - $100,000 plus Super + Bonus'",Sydney,Finite IT Recruitment Solutions,Information & Communication Technology,"b'\n<p><font face=""Calibri"" size=""2"">Data Ware...",90000100000,90000100000,95000,high,793
6,Data Steward - SAP / MDM,"b'$70,000 to $80,000 + super'",Sydney,Perigon Group Pty Limited,Information & Communication Technology,"b""Data Steward - SAP / MDM<br/>\n<br/>\r\nWe a...",7000080000,7000080000,75000,good,793
7,Data Engineer,0,Sydney,IAG,Information & Communication Technology,"b""\n<ul>\n<li>One of Australia's leading gener...",0,0,0,0,793
8,Data Scientist,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0,0,0,793
9,Data Analyst,0,Sydney,NRMA Motoring &amp; Services,Marketing & Communications,b'\n<p>The NRMA was born to keep people moving...,0,0,0,0,793


In [220]:
## save cleaned dataframe to new csv
data.to_csv('/Users/lohyenwei/Desktop/data_science/Projects/Project4/Project4_uploads/yen/clean_dataP4.csv')